In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('shipping_crew').getOrCreate()

23/01/02 09:45:40 WARN Utils: Your hostname, marcelo-Vostro-5470 resolves to a loopback address: 127.0.1.1; using 192.168.0.114 instead (on interface wlp8s0)
23/01/02 09:45:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/01/02 09:45:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from pyspark.ml.regression import LinearRegression

In [9]:
data = spark.read.csv("/home/marcelo/Documents/shipping_crew/cruise_ship_info.csv", header=True, inferSchema=True)

In [10]:
data.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Elation|   Carnival| 15|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|
|    Fantasy|   Carnival| 23| 

In [11]:
from pyspark.ml.feature import StringIndexer

In [13]:
indexer = StringIndexer(inputCol='Cruise_line', outputCol='Cruise_line_index')
indexed = indexer.fit(data).transform(data)
indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_line_index|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+-----------------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|             16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|              1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|              1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|              1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|       

In [15]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [23]:
assembler = VectorAssembler(
    inputCols=['Cruise_line_index', 'Age', 'Tonnage', 'passengers', 'length', 'cabins', 'passenger_density'],
    outputCol='features'
)

data_features = assembler.transform(indexed)
data_features.select('crew','features').show()

+----+--------------------+
|crew|            features|
+----+--------------------+
|3.55|[16.0,6.0,30.2769...|
|3.55|[16.0,6.0,30.2769...|
| 6.7|[1.0,26.0,47.262,...|
|19.1|[1.0,11.0,110.0,2...|
|10.0|[1.0,17.0,101.353...|
| 9.2|[1.0,22.0,70.367,...|
| 9.2|[1.0,15.0,70.367,...|
| 9.2|[1.0,23.0,70.367,...|
| 9.2|[1.0,19.0,70.367,...|
|11.5|[1.0,6.0,110.2389...|
|11.6|[1.0,10.0,110.0,2...|
| 6.6|[1.0,28.0,46.052,...|
| 9.2|[1.0,18.0,70.367,...|
| 9.2|[1.0,17.0,70.367,...|
| 9.3|[1.0,11.0,86.0,21...|
|11.6|[1.0,8.0,110.0,29...|
|10.3|[1.0,9.0,88.5,21....|
| 9.2|[1.0,15.0,70.367,...|
| 9.3|[1.0,12.0,88.5,21...|
| 9.2|[1.0,20.0,70.367,...|
+----+--------------------+
only showing top 20 rows



In [25]:
lr = LinearRegression(featuresCol='features', labelCol='crew', predictionCol='prediction')

In [24]:
train_data, test_data = data_features.randomSplit([0.7, 0.3])

In [26]:
model_train = lr.fit(train_data)

23/01/02 10:17:30 WARN Instrumentation: [326f16b2] regParam is zero, which might cause numerical instability and overfitting.


23/01/02 10:17:31 WARN InstanceBuilder$JavaBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.VectorBLAS


In [27]:
test_results = model_train.evaluate(test_data)

In [31]:
rmse = test_results.rootMeanSquaredError
mse = test_results.meanSquaredError
r2 = test_results.r2

In [32]:
print(rmse)
print(mse)
print(r2)

1.2524552785621013
1.5686442247980705
0.8495308821012324
